In [1]:
%pip install tables
%pip install hydrobr

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import glob
import os
from pathlib import Path
import h5py
from joblib import Parallel, delayed
import hydrobr

C:\Users\linde\AppData\Roaming\Python\Python311\site-packages\geopandas\_compat.py:154: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
replacement_map = {
    '├Ê': 'É',
    '├â': 'Ã',
    '├ü': 'Á',
    '├ç': 'Ç',
    '├ì': 'Í',
    '├Ò': 'é',
    '├│': 'ó',
    '├¡': 'í',
    '├ú': 'ã',
    '├í': 'á',
    '├º': 'ç',
    '├ô': 'Ó',
    '├õ': 'Ô',
    '├Ü': 'Ú',
    '├è': 'Ê',
    '├é': 'Â',
    '├┤': 'ô',
    '├║': 'ú',
    '├ò': 'õ',
    '├¬': 'ê',
    '├ó': 'â',
    '├╡': 'õ',
    '├Õ': 'Ú'
}

def fix_encoding(text, replacement_map):
    """
    Corrects encoding issues in a given text by replacing incorrectly encoded characters
    with their correct counterparts using a provided replacement map.

    Parameters:
        text (str): The input string that may contain incorrectly encoded characters.
        replacement_map (dict): A dictionary where keys are the incorrectly encoded
                                characters and values are the correct characters to
                                replace them with.

    Returns:
        str: The corrected text with all instances of incorrectly encoded characters
             replaced by their correct counterparts.

    Example:
        text = "This is a t├¬st with s├│me encoding probl├¡ms."
        corrected_text = fix_encoding(text, replacement_map)
        print(corrected_text)
        # Output: "This is a têst with sóme encoding problems."

    How It Works:
        1. Iterates over each key-value pair in the `replacement_map`.
        2. Replaces all occurrences of the incorrect character (key) in the input `text`
           with the correct character (value).
        3. Returns the corrected text after all replacements are made.

    Notes:
        - The function is case-sensitive, so ensure the keys in `replacement_map` match
          the exact incorrect characters in the text.
        - The `replacement_map` should be tailored to the specific encoding issues
          present in the text.
    """
    for incorrect, correct in replacement_map.items():
        text = text.replace(incorrect, correct)
    return text

In [5]:
# Function to read a single HDF5 file
def read_hdf_file(file_path, key_table):
    """
    Reads a single HDF5 file and returns the data.

    Parameters:
    file_path (str): The path to the HDF5 file.

    Returns:
    DataFrame: The data contained in the HDF5 file.
    """
    return pd.read_hdf(file_path, key=key_table)

# HIDROWEB

In [5]:
# hidroweb_folder_path = Path("./1 - Organized data gauge/HIDROWEB/")
hidroweb_folder_path = "/content/drive/MyDrive/QualiBRain/Scripts e Dados/1 - Organized data gauge/HIDROWEB/"
hidroweb_folder_path = r".\1 - Organized data gauge\HIDROWEB\\"
hidroweb_h5_files = glob.glob(os.path.join(hidroweb_folder_path, "*.h5"))
print("quantity of files:", len(hidroweb_h5_files), "\nexample:", hidroweb_h5_files[0])

IndexError: list index out of range

In [9]:
countries = ['BOLÍVIA', 'PERU', 'PARAGUAI', 'ARGENTINA', 'GUIANA FRANCESA', 'SURINAME']
df_all_gauges = hydrobr.get_data.ANA.list_prec_stations()
filt = ~df_all_gauges['State'].isin(countries)
df_hidroweb_gauge_info = df_all_gauges[filt]
df_hidroweb_gauge_info.rename(columns = {'NYD':'Years', 'MD':'Percentage_of_missing_data', 'N_YWOMD':'Years_without_gaps',
                     'YWMD':'Percentage_of_missing_years'}, inplace = True)
# df_hidroweb_gauge_info = df_hidroweb_gauge_info[(df_hidroweb_gauge_info['Responsible'] != 'CEMADEN')]
df_hidroweb_gauge_info.dropna(inplace=True)
df_hidroweb_gauge_info.reset_index(inplace=True, drop = True)
df_hidroweb_gauge_info

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,Years,Percentage_of_missing_data,Years_without_gaps,Percentage_of_missing_years
0,SALINÓPOLIS,00047000,2,32,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,1958/01/01,1964/12/31,7,25.0,0,100.0
1,SALINÓPOLIS,00047002,2,32,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,1977/12/09,2019/08/31,43,3.5,35,18.6
2,CURUÇA,00047003,2,32,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,1981/07/01,2019/07/31,39,2.4,29,25.6
3,PRIMAVERA,00047004,2,32,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,1982/02/18,2019/08/31,38,0.0,35,7.9
4,MARUDA,00047005,2,32,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,1989/08/21,2019/07/31,31,5.0,20,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11887,ÁGUA FRIA,08460003,2,14,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964,1996/03/02,2018/04/30,23,1.8,16,30.4
11888,UIRAMUTA,08460004,2,14,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664,2002/07/13,2019/07/31,18,2.3,12,33.3
11889,NOVA ESPERANÇA/MARCO BV-8,08461000,2,14,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,1984/03/23,2019/06/30,36,12.3,13,63.9
11890,MISSÃO AUARIS - JUSANTE,08464001,2,14,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,1995/04/01,2019/06/30,25,4.2,15,40.0


In [10]:
df_hidroweb_gauge_info[df_hidroweb_gauge_info['Code'] == '02243468']

,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude,StartDate,EndDate,Years,Percentage_of_missing_data,Years_without_gaps,Percentage_of_missing_years
8079,RIO DE JANEIRO_CMSVilaCanoas,02243468,2,59,RIO DE JANEIRO,RIO DE JANEIRO,CEMADEN,-22.9954,-43.2698,2015/07/22,2015/09/06,1,95.7,0,100.0


In [16]:
# prompt: rename df_gauge colums Name as name_station, State as state, Code as gauge_code, City as city,  Responsible as responsible, Latitude as lat, Longitude as long and drop other columns

df_hidroweb_gauge_info = df_hidroweb_gauge_info.rename(columns={
    'Name': 'name_station',
    'State': 'state',
    'Code': 'gauge_code',
    'City': 'city',
    'Responsible': 'responsible',
    'Latitude': 'lat',
    'Longitude': 'long'
})

# Drop other columns (replace with the actual column names you want to drop)
columns_to_drop = ['Type', 'SubBasin', 'StartDate', 'EndDate', 'Years', 'Percentage_of_missing_data', 'Years_without_gaps', 'Percentage_of_missing_years'] # Example
df_hidroweb_gauge_info = df_hidroweb_gauge_info.drop(columns=columns_to_drop, errors='ignore')
df_hidroweb_gauge_info['source'] = 'HIDROWEB'
df_hidroweb_gauge_info


,name_station,gauge_code,city,state,responsible,lat,long,source
0,SALINÓPOLIS,00047000,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,HIDROWEB
1,SALINÓPOLIS,00047002,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,HIDROWEB
2,CURUÇA,00047003,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,HIDROWEB
3,PRIMAVERA,00047004,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,HIDROWEB
4,MARUDA,00047005,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,HIDROWEB
...,...,...,...,...,...,...,...,...
11732,ÁGUA FRIA,08460003,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964,HIDROWEB
11733,UIRAMUTA,08460004,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664,HIDROWEB
11734,NOVA ESPERANÇA/MARCO BV-8,08461000,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,HIDROWEB
11735,MISSÃO AUARIS - JUSANTE,08464001,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,HIDROWEB


In [8]:
table_key = 'table_data'

# Use parallel processing to read all files
hidroweb_list_data = Parallel(n_jobs=-1)(delayed(read_hdf_file)(filename, table_key) for filename in hidroweb_h5_files)

# Concatenate and remove duplicates
# df_hidroweb_gauge_data = pd.concat(hidroweb_list_data).drop_duplicates(ignore_index=True)
df_hidroweb_gauge_data = pd.concat(hidroweb_list_data)
del hidroweb_list_data

df_hidroweb_gauge_data = df_hidroweb_gauge_data.rename(columns={
    'Date': 'datetime',
    'Value': 'rain_mm',
    'Code': 'gauge_code'
})

df_hidroweb_gauge_data

KeyError: 'datetime'

In [19]:
# Convert the 'event_date' column to datetime
df_hidroweb_gauge_data['datetime'] = pd.to_datetime(df_hidroweb_gauge_data['datetime'])

# Set 'datetime' as the index
df_hidroweb_gauge_data.set_index('datetime', inplace=True)

# Resample to daily frequency and aggregate 'rain_mm' using sum
# df_hidroweb_gauge_data = df_hidroweb_gauge_data.groupby(['gauge_code']).resample('D')['rain_mm'].sum().reset_index()


# Apply the function to the 'City' column

df_hidroweb_gauge_data['rain_mm'] = df_hidroweb_gauge_data['rain_mm'].astype('float64')
df_hidroweb_gauge_data['gauge_code'] = df_hidroweb_gauge_data['gauge_code'].astype('str')

df_hidroweb_gauge_data

In [22]:
df_hidroweb_gauge_data = df_hidroweb_gauge_data[
    (df_hidroweb_gauge_data['datetime'].dt.year >= 1961)
    & (df_hidroweb_gauge_data['datetime'].dt.year <= 2020)
    ].reset_index(drop = True).copy(deep=True)

df_hidroweb_gauge_data

In [20]:
df_hidroweb_gauge_data.dtypes

,0
gauge_code,object
datetime,datetime64[ns]
rain_mm,float64


In [24]:
# df_hidroweb_gauge_data.to_hdf('./1 - Organized data gauge/HIDROWEB_GAUGE_DATA_2021_2023.h5', key='table_data', mode='w', complevel=9, complib='zlib', encoding='utf-8')
df_hidroweb_gauge_data.to_hdf('HIDROWEB_DAILY_1961_2020.h5', key='table_data', mode='w', complevel=9, complib='zlib', encoding='utf-8')
del df_hidroweb_gauge_data

In [25]:
# df_hidroweb_gauge_info.to_hdf('./1 - Organized data gauge/HIDROWEB_GAUGE_DATA_2021_2023.h5', key='table_info', mode='r+', complevel=9, complib='zlib', encoding='utf-8')
df_hidroweb_gauge_info.to_hdf('HIDROWEB_DAILY_1961_2020.h5', key='table_info', mode='r+', complevel=9, complib='zlib', encoding='utf-8')
del df_hidroweb_gauge_info

In [28]:
df = pd.read_hdf('HIDROWEB_DAILY_1961_2020.h5', key='table_info')
df

,name_station,gauge_code,city,state,responsible,lat,long,source
0,SALINÓPOLIS,00047000,SALINÓPOLIS,PARÁ,INMET,-0.6500,-47.5500,HIDROWEB
1,SALINÓPOLIS,00047002,SALINÓPOLIS,PARÁ,ANA,-0.6231,-47.3536,HIDROWEB
2,CURUÇA,00047003,CURUÇA,PARÁ,ANA,-0.7375,-47.8536,HIDROWEB
3,PRIMAVERA,00047004,PRIMAVERA,PARÁ,ANA,-0.9294,-47.0994,HIDROWEB
4,MARUDA,00047005,MARAPANIM,PARÁ,ANA,-0.6336,-47.6583,HIDROWEB
...,...,...,...,...,...,...,...,...
11732,ÁGUA FRIA,08460003,UIRAMUTA,RORAIMA,ANA,4.6428,-60.4964,HIDROWEB
11733,UIRAMUTA,08460004,UIRAMUTA,RORAIMA,ANA,4.5986,-60.1664,HIDROWEB
11734,NOVA ESPERANÇA/MARCO BV-8,08461000,PACARAIMA,RORAIMA,ANA,4.4883,-61.1297,HIDROWEB
11735,MISSÃO AUARIS - JUSANTE,08464001,BOA VISTA,RORAIMA,ANA,4.0031,-64.4431,HIDROWEB
